### Initialization

In [22]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time
import pprint
from copy import deepcopy
if './src' not in sys.path:
    sys.path.insert(0, './src')
# print(sys.path)

import wandb
import numpy as np
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
from tqdm import tqdm, trange

from envs.sparsechem_env_dev import SparseChemEnv_Dev

from utils.sparsechem_utils import load_sparse, load_task_weights, class_fold_counts, fold_and_transform_inputs, print_metrics_cr
from dataloaders.chembl_dataloader_dev import ClassRegrSparseDataset_v3, ClassRegrSparseDataset, InfiniteDataLoader
from utils.util import ( makedir, print_separator, create_path, print_yaml, print_yaml2, print_loss, should, 
                         fix_random_seed, read_yaml, timestring, print_heading, print_dbg, 
                         print_underline, write_config_report, display_config, get_command_line_args, is_notebook) 
### Read Configuration File### Initialization
pp = pprint.PrettyPrinter(indent=4)
os.environ["WANDB_NOTEBOOK_NAME"] = "Adashare_Test.ipynb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
TEST_FROM_PATH  = '../experiments/AdaSparseChem/50x6_0225_1530_Retrain_0_plr0.01_sp0.0001_sh0.01'
TEST_MODEL_CKPT = 'best_retrain'
 

input_args = " --config yamls/chembl_3task_test.yaml " \
             " --exp_instance  0225_1530" \
             " --folder_sfx    test_0" \
             " --exp_desc Test phase of 00225_1530 run - 0" \
             " --exp_ids 1 "\
             " --batch_size 128".split()

### Read Configuration File
# get command line arguments
args = get_command_line_args(input_args)

    
print_underline(' command line parms : ', True)
for key, val in vars(args).items():
    print(f" {key:.<25s}  {val}")


  command line parms : 
------------------------
 config...................  yamls/chembl_3task_test.yaml
 exp_instance.............  0225_1530
 folder_sfx...............  test_0
 exp_desc.................  Test phase of 00225_1530 run - 0
 exp_ids..................  [1]
 batch_size...............  128
 backbone_lr..............  None
 task_lr..................  None
 decay_lr_rate............  None
 decay_lr_freq............  None
 gpus.....................  [0]
 cpu......................  False


In [30]:
# ********************************************************************
# ****************** create folders and print options ****************
# ********************************************************************
print_separator('READ YAML')

opt, gpu_ids, exp_ids = read_yaml(args)

exp_id = exp_ids[0]
fix_random_seed(opt["seed"][exp_id])
    
create_path(opt)    
print()
print_heading(f" experiment name       : {opt['exp_name']} \n"
              f" experiment instance   : {opt['exp_instance']} \n"
              f" folder_name           : {opt['exp_folder']} \n"
              f" folder_sfx            : {opt['folder_sfx']} \n"
              f" experiment description: {opt['exp_description']}\n"
              f" log folder            : {opt['paths']['log_dir']}\n"
              f" checkpoint folder     : {opt['paths']['checkpoint_dir']}", verbose = True)
print(f" Gpu ids: {gpu_ids}     exp_ids: {exp_ids}         seed: {opt['seed']}")        

write_config_report(opt, filename = 'run_params.txt')    
display_config(opt)        

##################################################
################### READ YAML ####################
##################################################
 Create  log_dir  folder:  ../experiments/AdaSparseChem/50x6_0225_1530_test_0
 Create  result_dir  folder:  ../experiments/AdaSparseChem/50x6_0225_1530_test_0
 Create  checkpoint_dir  folder:  ../experiments/AdaSparseChem/50x6_0225_1530_test_0

------------------------------------------------------------------------------------------------------------------------
 experiment name       : SparseChem 
 experiment instance   : 0225_1530 
 folder_name           : 50x6_0225_1530_test_0 
 folder_sfx            : test_0 
 experiment description: Test phase of 00225_1530 run - 0
 log folder            : ../experiments/AdaSparseChem/50x6_0225_1530_test_0
 checkpoint folder     : ../experiments/AdaSparseChem/50x6_0225_1530_test_0
---------------------------------------------------------------------------------------------------------------------

In [31]:
# ********************************************************************
# ******************** Prepare the dataloaders ***********************
# ********************************************************************
# load the dataloader
print_separator('CREATE DATALOADERS')
print(opt['dataload']['x_split_ratios'])

if opt['dataload']['dataset'] == 'Chembl_23_mini':
    testset  = ClassRegrSparseDataset_v3(opt, split_ratios = opt['dataload']['x_split_ratios'], ratio_index = 2, verbose = False)
    print('size of test set  : ', len(testset))
else:
    raise NotImplementedError('Dataset %s is not implemented' % opt['dataload']['dataset'])

print('size of test set: ', len(testset))

# test_loader = DataLoader(testset, batch_size=opt['test']['batch_size'], drop_last=True, num_workers=2, shuffle=True)
#     val_loader = DataLoader(valset, batch_size=1, drop_last=True, num_workers=2, shuffle=False)

# test_loader = DataLoader(testset, batch_size=1, drop_last=True, num_workers=2, shuffle=True)
test_loader   = InfiniteDataLoader(testset, batch_size=1, num_workers = 1, pin_memory=True, collate_fn=testset.collate  , shuffle=False)


# ********************************************************************
# ********************Create the environment *************************
# ********************************************************************
# create the model and the pretrain model
print_separator('CREATE THE ENVIRONMENT')
environ = SparseChemEnv_Dev(log_dir          = opt['paths']['log_dir'], 
                            checkpoint_dir   = opt['paths']['checkpoint_dir'], 
                            exp_name         = opt['exp_name'],
                            tasks_num_class  = opt['tasks_num_class'], 
#                                 init_neg_logits  = opt['train']['init_neg_logits'], 
                            device           = gpu_ids[0],
#                                 init_temperature = opt['train']['init_temp'], 
#                                 temperature_decay= opt['train']['decay_temp'], 
                            is_train         = False,
                            opt              = opt, 
                            verbose          = False)

cfg = environ.print_configuration()
# print(cfg)
write_config_report(opt, cfg, filename = 'run_params.txt', mode = 'a')    

    
    # create the model and the pretrain model
#     print_separator('CREATE THE ENVIRONMENT')
#     environ = BlockDropEnv(opt['paths']['log_dir'], 
#                            opt['paths']['checkpoint_dir'], 
#                            opt['exp_name'],
#                            opt['tasks_num_class'],
#                            device=gpu_ids[0], 
#                            is_train=False, 
#                            opt=opt)

##################################################
############### CREATE DATALOADERS ###############
##################################################
[0.725, 0.225, 0.05]
size of test set  :  920
size of test set:  920
##################################################
############# CREATE THE ENVIRONMENT #############
##################################################
-------------------------------------------------------
* SparseChemEnv_Dev  Initializtion - verbose: False
------------------------------------------------------- 

------------------------------------------------------------
SparseChemEnv_Dev.super() init()  Start - verbose: False
------------------------------------------------------------ 

 log_dir        :  ../experiments/AdaSparseChem/50x6_0225_1530_test_0 
 checkpoint_dir :  ../experiments/AdaSparseChem/50x6_0225_1530_test_0 
 exp_name       :  SparseChem 
 tasks_num_class:  [5, 5, 5] 
 device         :  cuda:0 
 device id      :  0 
 dataset        :  Chembl_

In [32]:
##
## Load Model and Policy snapshots
##
# current_iter = environ.load_checkpoint('retrain%03d_policyIter%s_best' % (exp_ids[0], opt['train']['policy_iter']))

print_underline(f"\n 1. load checkpoint : {TEST_FROM_PATH} - {TEST_MODEL_CKPT}", verbose = True)
current_iter = environ.load_checkpoint(TEST_MODEL_CKPT, path=TEST_FROM_PATH, verbose = True)

print_underline('Evaluating the snapshot saved at %d iter' % current_iter, verbose = True)

# policy_label = 'Iter%s_rs%04d' % (opt['train']['policy_iter'], opt['seed'][exp_ids[0]])
policy_label = 'Iter_%s_expid_%02d_seed_%04d' % (opt['train']['policy_iter'], exp_id, opt['seed'][exp_id])
print_underline(f"\n A. load existing POLICY: {policy_label}", verbose = True)

if environ.check_exist_policy(policy_label,path = TEST_FROM_PATH):
    environ.load_policy(policy_label, path = TEST_FROM_PATH, verbose = True)
else:
    print(f"{policy_label} not found ")

policys = environ.get_current_policy()
overall_policy = np.concatenate(policys, axis=-1)
print_underline('loaded policy ', verbose = True)
print(overall_policy)


 
 1. load checkpoint : ../experiments/AdaSparseChem/50x6_0225_1530_Retrain_0_plr0.01_sp0.0001_sh0.01 - best_retrain
---------------------------------------------------------------------------------------------------------------------
=> loading snapshot from ../experiments/AdaSparseChem/50x6_0225_1530_Retrain_0_plr0.01_sp0.0001_sh0.01/best_retrain_model.pth.tar
   Loading to CPU
  networks -  network:  mtl-net
  load snapshot - network:  mtl-net
    network mtl-net - item task1_logits
    network mtl-net - item task2_logits
    network mtl-net - item task3_logits
    network mtl-net - item backbone.Input_linear.weight
    network mtl-net - item backbone.Input_linear.bias
    network mtl-net - item backbone.blocks.0.0.linear.weight
    network mtl-net - item backbone.blocks.0.0.linear.bias
    network mtl-net - item backbone.blocks.1.0.linear.weight
    network mtl-net - item backbone.blocks.1.0.linear.bias
    network mtl-net - item backbone.blocks.2.0.linear.weight
    network mtl-n

AttributeError: 'SparseChemEnv_Dev' object has no attribute 'policy1'

In [8]:
if torch.cuda.is_available():
    environ.cuda(gpu_ids)

    # ********************************************************************
    # ******************************  Test  ******************************
    # ********************************************************************
#     num_seg_class = opt['tasks_num_class'][opt['tasks'].index('seg')] if 'seg' in opt['tasks'] else -1
    environ.eval()
    
    
#     val_metrics = eval_fix_policy(environ, val_loader, opt['tasks'], num_seg_cls=num_seg_class, eval_iter=-1)
    val_metrics = environ.evaluate(test_loader, 
                                   is_policy=True,
                                   policy_sampling_mode = 'fix_policy')
#                                    hard_sampling=None,  ## hard_sampling=opt['train']['hard_sampling'],
#                                    eval_iters = -1, 
#                                    progress = True, 
#                                    leave = False, 
#                                    verbose = False)    
    print(val_metrics)

eval iters set to :  920


None


In [17]:
print(environ.val_metrics['task1']['classification'])
print(environ.val_metrics['task2']['classification'])
print(environ.val_metrics['task3']['classification'])

      roc_auc_score    auc_pr  avg_prec_score    f1_max  p_f1_max     kappa  \
task                                                                          
0          0.809947  0.870241        0.870419  0.809683  0.234723  0.454949   
1          0.702792  0.668633        0.669218  0.645374  0.201933  0.319945   
2          0.790758  0.759362        0.760180  0.773430  0.032953  0.435676   
3          0.790516  0.740017        0.740502  0.678700  0.482500  0.411142   
4          0.700168  0.759219        0.759621  0.719298  0.000054  0.291268   

      kappa_max  p_kappa_max   bceloss  
task                                    
0      0.472036     0.745512  0.776153  
1      0.349626     0.645575  1.607327  
2      0.490710     0.146744  1.025560  
3      0.458359     0.818019  0.916853  
4      0.330854     0.896638  1.467913  
      roc_auc_score    auc_pr  avg_prec_score    f1_max  p_f1_max     kappa  \
task                                                                          
0

In [18]:
pp.pprint(environ.val_metrics)
# pp.pprint(environ.val_metrics)
# print(val_metrics)

{   'aggregated': {   'auc_pr': 0.7839680874062291,
                      'avg_prec_score': 0.7844142007773247,
                      'bceloss': 1.0481916864713035,
                      'f1_max': 0.7444299731015749,
                      'kappa': 0.4173461494995934,
                      'kappa_max': 0.4453778160823674,
                      'logloss': 0.001139338805300659,
                      'p_f1_max': 0.1728429940039253,
                      'p_kappa_max': 0.5572595953941344,
                      'roc_auc_score': 0.7797019318818047,
                      'sc_loss': 0.017090082079509884},
    'losses': {   'task1': 5.793805089180847,
                  'task2': 4.7581341425005,
                  'task3': 5.170936281467743,
                  'total': 15.722875513149093},
    'losses_mean': {   'task1': 1.1587610178361685,
                       'task2': 0.9516268285000998,
                       'task3': 1.0341872562935486,
                       'total': 3.144575102629818},
    

In [20]:
environ.save_metrics(policy_label, verbose = True)

 save_policy(): save metrics to ../experiments/AdaSparseChem/50x6_0225_1530_test_0/metrics_Iter_best_expid_00_seed_0088.pickle


In [ ]:
if __name__ == "__main__":
    test()

### Functions

In [ ]:
from utils.util import init_records, populate_records, populate_validation_metrics

def eval_fix_policy(environ, dataloader, tasks, num_seg_cls=-1, eval_iter=10):
    batch_size = []
    records = {}
    val_metrics = {}

    records = init_records(tasks, num_seg_cls)
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(dataloader)):
            if eval_iter != -1:
                if batch_idx > eval_iter:
                    break
            environ.set_inputs(batch)
            
            metrics = environ.val_fix_policy()
            
            populate_records(records, metrics, tasks)

            batch_size.append(len(batch['img']))

    ##  Populate Validation Metrics 
    val_metrics = populate_validation_metrics(records, tasks, num_seg_cls)

    return val_metrics